# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import os

import numpy as np
import pandas as pd
import re

import nltk
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.corpus.reader.wordnet import WordNetError

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Victoria\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Victoria\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Victoria\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Victoria\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
pth = r'C:\Users\Victoria\anaconda\envs\ds_practicum_env\toxic_comments.csv'

pth1 = '/datasets/toxic_comments.csv'

if os.path.exists(pth):
    data = pd.read_csv(pth)
elif os.path.exists(pth1):
    data = pd.read_csv(pth1)
else:
    print('Something is wrong')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
data['text'].duplicated().sum()

0

In [5]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [6]:
class_ratio = data['toxic'].value_counts()[0] / data['toxic'].value_counts()[1]
class_ratio

8.841344371679229

In [7]:
#удаляю столбец Unnamed:
data = data.drop(['Unnamed: 0'], axis=1)

In [8]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [9]:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = ' '.join(text.split())
    return text

data['text'] = data['text'].apply(clear_text) 

In [11]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,               #прилагательное
                "N": wordnet.NOUN,              #существительное
                "V": wordnet.VERB,              #глагол
                "R": wordnet.ADV                #наречие
               }  
    return tag_dict.get(tag, wordnet.NOUN)

In [12]:
lemmatizer = WordNetLemmatizer()

#ввожу функцию леммализации тектов постов:
def lemmatize_text(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

data['lemm_text'] = data['text'].apply(lemmatize_text)

data = data.drop(['text'], axis=1)

In [13]:
data.head()

,toxic,lemm_text
0,0,explanation why the edits make under my userna...
1,0,d aww he match this background colour i m seem...
2,0,hey man i m really not try to edit war it s ju...
3,0,more i can t make any real suggestion on impro...
4,0,you sir be my hero any chance you remember wha...


In [18]:
corpus = data['lemm_text'].values.astype('str')

In [19]:
stopwords = set(nltk_stopwords.words('english'))

In [20]:
count_vect = CountVectorizer()
bow_no_stop = count_vect.fit_transform(corpus) 

# создайте мешок слов без учёта стоп-слов
# < напишите код здесь >

print("Размер мешка без учёта стоп-слов:", bow_no_stop.shape)

count_vect = CountVectorizer(stop_words=stopwords)
bow_stop = count_vect.fit_transform(corpus) 

# создайте новый мешок слов с учётом стоп-слов
# < напишите код здесь >

print("Размер мешка с учётом стоп-слов:", bow_stop.shape)

Размер мешка без учёта стоп-слов: (159292, 151900)
Размер мешка с учётом стоп-слов: (159292, 151766)


In [21]:
tf_idf = count_vect.fit_transform(corpus)

In [22]:
print("Размер матрицы:", tf_idf.shape)

Размер матрицы: (159292, 151766)


Получили матрицу со значениями TF-IDF. Признаков столько же, сколько было у мешка слов.

**Выводы:**

- Удалила ненужный столбец
- Леммализировала тексты, убрала стоп-слова

## Обучение

In [28]:
#разделяю выборки в соотношении 80/20:

train, test = train_test_split(data, test_size=0.2, random_state=12345)

features_train = train.drop(['toxic'], axis=1) 
target_train = train['toxic']

features_test = test.drop(['toxic'], axis=1) 
target_test = test['toxic']

In [29]:
for i in [features_train, target_train, features_test, target_test]:
    print(i.shape)

(127433, 1)
(127433,)
(31859, 1)
(31859,)


In [30]:
features_train = features_train['lemm_text']

### Модель дерева решений

In [31]:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("dtc", DecisionTreeClassifier())])

In [32]:
parameters = {'dtc__max_depth': ([x for x in range(1, 25)]),
              'dtc__class_weight': (['balanced'])}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
dtc_train_f1 = max(mts)

print('F1 дерева решений =', round(dtc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

F1 дерева решений = 0.63
при параметрах {'dtc__class_weight': 'balanced', 'dtc__max_depth': 24}



Метрика F1 хуже необходимого значения по условию проекта (не менее 0.75). Модель не подходит, рассмотрим другую модель.

### Модель логистической регрессии LogisticRegression

In [34]:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True)), 
                     ("lr", LogisticRegression())])

In [35]:
#обучение:
    
parameters = {'lr__C': (.1, 1, 5, 10),
              'lr__max_iter': ([200]),
              'lr__class_weight': (['balanced'])}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
lr_train_f1 = max(mts)

print('F1 логистической регрессии =', round(lr_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

F1 логистической регрессии = 0.76
при параметрах {'lr__C': 5, 'lr__class_weight': 'balanced', 'lr__max_iter': 200}



**Вывод:** при сравнении метрики F1 разных моделей на кросс-валидации модель логистической регрессии показывает лучше результат, чем модель дерева решений

### Расчет метрики F1 лучшей модели на тестовой выборке

In [39]:
#тестирование:
predictions_test = gscv.predict(features_test['lemm_text'])
lr_test_f1 = f1_score(target_test, predictions_test)
print('финальный F1 логистической регрессии =', round(lr_test_f1,2))
print('полнота:',round(recall_score(target_test,predictions_test),2))
print('точность:', round(precision_score(target_test,predictions_test),2))

финальный F1 логистической регрессии = 0.76
полнота: 0.84
точность: 0.7


**Вывод:** F1 выше необходмого условия не менее 0.75. Модель подходит

## Выводы

Модель логистической реграссии показала лучшие результаты, чем модель логистической регрессии. 
Модель логистической реграссии показала результат лучше, чем модель логистической регрессии. F1 не менее 0.77. 
В этой метрике учтены как полнота так и точность предсказания. 
Модель хорошо предсказывает положительные классы (токсичные комментарии), поскольку высока как точность (0.7), так и полнота (0.84).


## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны